In [1]:
import tensorflow as tf
from sklearn.datasets import fetch_mldata
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix

In [2]:
# Python optimisation variables
learning_rate = 0.01
epochs = 2000
batch_size = 256

In [3]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
newsgroups_train = fetch_20newsgroups(subset="train")
from sklearn.feature_extraction import stop_words
newsgroups_trainlabel = newsgroups_train.target
vectorizer = TfidfVectorizer(stop_words=stop_words.ENGLISH_STOP_WORDS)
vectorstrain = vectorizer.fit_transform(newsgroups_train.data)
newsgroups_test = fetch_20newsgroups(subset="test")
vocab = vectorizer.vocabulary_
newvectorizer = TfidfVectorizer(vocabulary=vocab)
vectorstest = newvectorizer.fit_transform(newsgroups_test.data)
newsgroups_testlabel = newsgroups_test.target

In [4]:
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest
print(vectorstrain.shape)
X_new = SelectKBest(chi2, k=1000).fit(vectorstrain, newsgroups_trainlabel)
chi2result = np.where(X_new.get_support())[0]
vectorstestchi2 = vectorstest[:,chi2result]
print(vectorstestchi2.shape)
vectorstrainchi2 = vectorstrain[:,chi2result]
print(vectorstrainchi2.shape)

(11314, 129796)
(7532, 1000)
(11314, 1000)


In [5]:
hot_vector_label_train = np.zeros((newsgroups_trainlabel.shape[0],20))
print(hot_vector_label_train.shape)
index = 0
for label in newsgroups_trainlabel:
    #print(label)
    hot_vector_label_train[index][int(label)] = 1
    index += 1

print(hot_vector_label_train)

index = 0

hot_vector_label_test = np.zeros((newsgroups_testlabel.shape[0],20))
print(hot_vector_label_test.shape)
index = 0
for label in newsgroups_testlabel:
    #print(label)
    hot_vector_label_test[index][int(label)] = 1
    index += 1

print(hot_vector_label_test)

(11314, 20)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(7532, 20)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [6]:
vectorstrain.shape

(11314, 129796)

In [7]:
def randomn_selection(X_train,label_train,k):
    idx = np.random.randint(X_train.shape[0], size=k)
    return X_train[idx],label_train[idx]

In [13]:

x = tf.placeholder(tf.float32, [None, vectorstrainchi2.shape[1]], name='InputData')
# Documents => 20 classes
y = tf.placeholder(tf.float32, [None, 20], name='LabelData')

# now declare the weights connecting the input to the hidden layer
print(vectorstrainchi2.shape[1])
W1 = tf.Variable(tf.random_normal([vectorstrainchi2.shape[1], 32],stddev=0.01), name='W1')
b1 = tf.Variable(tf.random_normal([32]), name='b1')

# and the weights connecting the hidden layer to the output layer
W2 = tf.Variable(tf.random_normal([32, 20],stddev=0.1), name='W3')
b2 = tf.Variable(tf.random_normal([20]), name='b3')

hidden_out = tf.add(tf.matmul(x, W1), b1)
hidden_out = tf.nn.relu(hidden_out)

1000


In [14]:
with tf.name_scope('Model'):
    # Model
    pred = tf.nn.softmax(tf.add(tf.matmul(hidden_out, W2), b2))
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    cost = -tf.reduce_mean(tf.reduce_sum(y * tf.log(pred)+ (1 - y) * tf.log(1 - pred), axis=1))
with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    # define an accuracy assessment operation
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(pred, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [15]:
# finally setup the initialisation operator
init_op = tf.global_variables_initializer()

In [16]:
vectorstrainchi2.shape

(11314, 1000)

In [17]:
sess = tf.Session()
sess.run(init_op)
total_batch = int(vectorstrainchi2.todense().shape[0] / batch_size)
print(total_batch)
for epoch in range(epochs):
    avg_cost = 0
    for i in range(total_batch):
        batch_x, batch_y = randomn_selection(vectorstrainchi2.todense(), hot_vector_label_train,batch_size)
        l, c = sess.run([optimizer, cost], feed_dict={x:batch_x , y:batch_y})   
    if(epoch % 100 == 0):
        print('Step %i: Minibatch Loss: %f' % (epoch, c))
        print(sess.run(accuracy, feed_dict={x: vectorstestchi2.todense(), y: hot_vector_label_test}))

44
Step 0: Minibatch Loss: 4.249251
0.041157726
Step 100: Minibatch Loss: 3.950016
0.098114714
Step 200: Minibatch Loss: 3.937948
0.0893521
Step 300: Minibatch Loss: 3.927655
0.17817312
Step 400: Minibatch Loss: 3.892547
0.2940786
Step 500: Minibatch Loss: 3.854112
0.38568774
Step 600: Minibatch Loss: 3.746082
0.42485395
Step 700: Minibatch Loss: 3.642979
0.44941583
Step 800: Minibatch Loss: 3.433110
0.4731811
Step 900: Minibatch Loss: 3.117323
0.5207116
Step 1000: Minibatch Loss: 2.871674
0.57116306
Step 1100: Minibatch Loss: 2.782899
0.59121084
Step 1200: Minibatch Loss: 2.563456
0.62360597
Step 1300: Minibatch Loss: 2.206447
0.6419278
Step 1400: Minibatch Loss: 2.177583
0.65932024
Step 1500: Minibatch Loss: 1.911677
0.6735263
Step 1600: Minibatch Loss: 1.874744
0.68414766
Step 1700: Minibatch Loss: 1.749049
0.6950345
Step 1800: Minibatch Loss: 1.624669
0.7047265
Step 1900: Minibatch Loss: 1.418243
0.7100372
Step 2000: Minibatch Loss: 1.364629
0.7126925
Step 2100: Minibatch Loss: 1.5

KeyboardInterrupt: 